<a href="https://colab.research.google.com/github/vu-topics-in-big-data-2022/examples/blob/main/example-kafka/Kafka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Read https://towardsdatascience.com/kafka-python-explained-in-10-lines-of-code-800e3e07dad1

In [1]:
#zookeeper is packaged in a debian package that you can install
#kazoo is the python library used to work with zookeeper
!apt-get update
!apt-get install zookeeperd
!pip install kazoo

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:13 

In [4]:
#install Java
!apt-get install openjdk-8-jdk-headless
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [5]:
#get kafka
!wget  https://archive.apache.org/dist/kafka/3.1.0/kafka_2.13-3.1.0.tgz
!tar -zxf kafka_2.13-3.1.0.tgz
!sudo mv kafka_2.13-3.1.0 /usr/local/kafka
!sudo mkdir -p /tmp/kafka-logs

--2022-04-12 16:22:33--  https://archive.apache.org/dist/kafka/3.1.0/kafka_2.13-3.1.0.tgz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88130011 (84M) [application/x-gzip]
Saving to: ‘kafka_2.13-3.1.0.tgz’

kafka_2.13-3.1.0.tg 100%[===================>]  84.05M  13.2MB/s    in 6.7s    

2022-04-12 16:22:40 (12.5 MB/s) - ‘kafka_2.13-3.1.0.tgz’ saved [88130011/88130011]



In [6]:
# We start the zookeeper service
!service zookeeper start

In [7]:
# Start the kafka server
!/usr/local/kafka/bin/kafka-server-start.sh -daemon /usr/local/kafka/config/server.properties

In [10]:
# Create a new "topic"
!/usr/local/kafka/bin/kafka-topics.sh --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic bigdata

Created topic bigdata.


In [11]:
#verify the topics
!/usr/local/kafka/bin/kafka-topics.sh --bootstrap-server localhost:9092  --describe --topic bigdata

Topic: bigdata	TopicId: p6GkM2DsTWKaDVrNvp0Y3w	PartitionCount: 1	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: bigdata	Partition: 0	Leader: 0	Replicas: 0	Isr: 0


In [12]:
#produce a message 
# Type in messages and press enter.
# When done, manually stop the cell
!/usr/local/kafka/bin/kafka-console-producer.sh --bootstrap-server localhost:9092 --topic bigdata

>here
>this
>test
>

In [13]:
#consume messages
# manually stop to continue, otherwise the consumer will keep on waiting for new
# messages.
!/usr/local/kafka/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic bigdata --from-beginning

here
this
test
Processed a total of 3 messages


In [14]:
#let us now try to write a simple python program to produce and consume messages.
!pip3 install kafka-python
# Create a new "topic" for python script purposes
!/usr/local/kafka/bin/kafka-topics.sh --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic pybigdata

     |████████████████████████████████| 246 kB 4.8 MB/s 
Created topic pybigdata.


In [15]:
#kafka Producer
from time import sleep
from json import dumps
from kafka import KafkaProducer
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))
for e in range(10):
    data = {'number' : e}
    producer.send('pybigdata', value=data)
    print("sent {}".format(data))
    sleep(2)


sent {'number': 0}
sent {'number': 1}
sent {'number': 2}
sent {'number': 3}
sent {'number': 4}
sent {'number': 5}
sent {'number': 6}
sent {'number': 7}
sent {'number': 8}
sent {'number': 9}


In [16]:
#so here is a basic consumer as well
from kafka import KafkaConsumer
from json import loads
consumer = KafkaConsumer(
    'pybigdata',
     bootstrap_servers=['localhost:9092'],
     consumer_timeout_ms=1000,#important for stopping if there is no data
     auto_offset_reset='earliest',
     value_deserializer=lambda x: loads(x.decode('utf-8')))
for message in consumer:
    message = message.value   
    print('got {}'.format(message))

got {'number': 0}
got {'number': 1}
got {'number': 2}
got {'number': 3}
got {'number': 4}
got {'number': 5}
got {'number': 6}
got {'number': 7}
got {'number': 8}
got {'number': 9}
